# Large Data Sets in Python

This is the companion source for the article [Large Data Sets in Python: Pandas and the Alternatives](https://codesolid.com/large-data-sets-in-python-pandas-and-the-alternatives/).  Note that this Notebook relies on the [Riiid Test Answer Prediction Data Set](https://www.kaggle.com/c/riiid-test-answer-prediction), which you'll need to download and unzip into the ./data directory.

The sections in this Notebook don't exactly match the sequence of the article, because we we've tried to keep similar tools together.  So here we'll run some operations in Pandas first, then Dask, then Polars. Note that the file size of train.db is large (about 5.4 Gb), so keep in mind:

* You'll need a machine with enough memory.  At least 16 Gb is recommended.
* You might periodically restart the kernel (Kernel / Restart Kernel from the menu) to free up memory, though since we re-use the "df" object, this isn't strictly necessary.

## Important:

This notebook compares several techniques that take time to execute.  Some of these cells will take 30-35 seconds or more to run so please be patient; others are significantly faster.

## Pandas

***
Reading from a CSV file into memory, no types specified.

In [1]:
from timer import Timer
import pandas as pd
df = None
with Timer("Pandas plain read_csv"):
    df = pd.read_csv("data/train.csv")

[Pandas plain read_csv]
Elapsed time: 36.61 seconds


***
Reading from a CSV file into memory, specifying types in advance.

In [1]:
import pandas as pd
from timer import Timer
import numpy as np

dtypes = {
'row_id': np.int64,
 'timestamp': np.int32, 
 'user_id': np.int32, 
 'content_id': np.int32, 
 'content_type_id': np.int32,
'task_container_id': np.int32, 
'user_answer': np.int32, 
'answered_correctly': np.int32,
'prior_question_elapsed_time': np.float64, 
'prior_question_had_explanation': object
}

with Timer("Read csv using polars"):
    df = pd.read_csv("data/train.csv", dtype=dtypes)

[Read csv using polars]
Elapsed time: 37.31 seconds


***
Set an index and fetch rows once in memory

In [2]:
with Timer("Pandas set_index"):
    df.set_index("user_id")

with Timer("Fetchs"):
    rows = df.loc[40828]

[Pandas set_index]
Elapsed time: 3.99 seconds
[Fetchs]
Elapsed time: 0.00 seconds


***
# Dask

Read the CSV file and set an index

In [4]:
from dask import dataframe
from timer import Timer

import numpy as np
dtypes = {
'row_id': np.int64,
 'timestamp': np.int32, 
 'user_id': np.int32, 
 'content_id': np.int32, 
 'content_type_id': np.int32,
'task_container_id': np.int32, 
'user_answer': np.int32, 
'answered_correctly': np.int32,
'prior_question_elapsed_time': np.float64, 
'prior_question_had_explanation': object
}

with Timer("dask read_csv"):
    df = dataframe.read_csv("data/train.csv", low_memory=False)
    
with Timer("set_index.csv"):
    df = df.set_index("user_id")

[dask read_csv]
Elapsed time: 0.58 seconds
[set_index.csv]
Elapsed time: 33.08 seconds


***
Select a set of rows with the index set:

In [6]:
with Timer("Get rows for user_id 40828"):
    rows = df.loc[40828].compute()

[Get rows for user_id 40828]
Elapsed time: 30.31 seconds


***
# Polars (Full Read -- Non-Lazy)

Read whole CSV in memory in Polars and select rows.

In [8]:
import polars as pl

with Timer("Read CSV into memory using Polars"):
    df = pl.read_csv("data/train.csv")

with Timer("Select rows in memory, polars"):
    rows = df.filter(pl.col('user_id') == 2147470777)

[Read CSV into memory using Polars]
Elapsed time: 13.99 seconds
[Select rows in memory, polars]
Elapsed time: 0.18 seconds


***

# Polars and Pandas using Parquet

Use Polars to read file (CSV) and write it back out to Parquet.  We later read it back in using Pandas.  The next two cells should be run in order.  The first one will take the longest, but it sets things up for better performance in the second cell.

In [13]:
import polars as pl
import pandas as pd

from timer import Timer

with Timer("Read CSV in Polars"):
    df = pl.read_csv("data/train.csv")

with Timer("Write to Parquet in Polars"):
    df.write_parquet("data/train.parquet")

[Read CSV in Polars]
Elapsed time: 15.97 seconds
[Write to Parquet in Polars]
Elapsed time: 18.97 seconds


In [14]:
# Read from Parquet in Polars and Pandas
df = None
with Timer("Time to read from parquet in Pandas"):
    df = pd.read_parquet("data/train.parquet")

df = None
with Timer("Time to read from parquet in Polars"):
    df = pl.read_parquet("data/train.parquet")

[Time to read from parquet in Pandas]
Elapsed time: 7.36 seconds
[Time to read from parquet in Polars]
Elapsed time: 5.38 seconds


***
# Polars Lazy Mode

In [16]:
import polars as pl
from timer import Timer 

with Timer("Scan CSV for early user (Polars)"):
    df = pl.scan_csv("data/train.csv").filter(pl.col('user_id') == 40828).collect()

with Timer("Read CSV for later user (Polars)"):
    df = pl.scan_csv("data/train.csv").filter(pl.col('user_id') == 2147470777).collect()


[Scan CSV for early user (Polars)]
Elapsed time: 3.64 seconds
[Read CSV for later user (Polars)]
Elapsed time: 3.52 seconds
